In [7]:
import utilities.light_generator as ulg
import msprime


MU_HUMAN = ulg.give_mu()*100
RHO_HUMAN = MU_HUMAN * ulg.give_random_coeff()
POPULATION = 500
num_replicates = 10
lengt = int(3e7)
is_experement = True

dg = ulg.DataGenerator(
    recombination_rate=RHO_HUMAN,
    mutation_rate=MU_HUMAN,
    sample_size=100,
    demographic_events=ulg.generate_demographic_events(POPULATION),
    num_replicates=num_replicates,
    lengt=lengt,
    random_seed=42,
    is_experement=is_experement)
dg.run_simulation()

<generator object _replicate_generator at 0x148257900>

In [8]:
%%time
data = next(dg)

CPU times: user 2min 40s, sys: 460 ms, total: 2min 41s
Wall time: 2min 41s


In [9]:
x, y, time = data

In [10]:
with open("42.txt","w+") as file:
    file.write("".join(map(str, x)))

In [13]:
" ".join(map(str, [1,2,3]))

'1 2 3'

In [11]:
import utilities.light_generator as ulg
import msprime

POPULATION = 500
num_replicates = 10
lengt = int(3e7)
is_experement = True
i = 0

for seed in range(11,1):
    MU_HUMAN = ulg.give_mu()
    RHO_HUMAN = ulg.give_mu() * ulg.give_random_coeff()
    
    dg = ulg.DataGenerator(
    recombination_rate=RHO_HUMAN,
    mutation_rate=MU_HUMAN,
    sample_size=100,
    demographic_events=ulg.generate_demographic_events(POPULATION),
    num_replicates=num_replicates,
    lengt=lengt,
    random_seed=42,
    is_experement=is_experement)
    
    for x, y, times in dg:
        with open(f"x/{i}.txt","w+") as file:
            file.write("".join(map(str, x)))
        with open(f"y/{i}.txt","w+") as file:
            file.write("".join(map(str, y)))
        with open(f"times/{i}.txt","w+") as file:
            file.write("".join(map(str, times)))

SyntaxError: unexpected EOF while parsing (<ipython-input-11-ed21879b384f>, line 22)